In [97]:
from rdflib import Graph, URIRef, RDF, Namespace, Literal
from rdflib.namespace import XSD
import pandas as pd
import os

from collections import namedtuple

OntologyData = namedtuple("OntologyData", [
    "source_path","target_path","source_graph", "target_graph",
    "source", "class_source", "op_source", "dp_source",
    "target", "class_target", "op_target", "dp_target"
])


def reload_source_target():

    # Dateipfade zur Quell- und Zielontologie
    source_ontology_path = r"C:\Users\Alexander Verkhov\Downloads\KB1_v0.5.ttl"
    target_ontology_path = r"C:\Users\Alexander Verkhov\Downloads\KB2_Appr2V0.1.ttl"

    # Überprüfen, ob die Dateien existieren
    if not os.path.exists(source_ontology_path):
        raise FileNotFoundError(f"Die Datei {source_ontology_path} wurde nicht gefunden.")
    if not os.path.exists(target_ontology_path):
        raise FileNotFoundError(f"Die Datei {target_ontology_path} wurde nicht gefunden.")

    # Namespaces der Quell- und Zielontologie
    source ="http://www.semanticweb.org/SkillOA/2025/3/KBI/"
    class_source = source + "class_"
    op_source = source + "op_"
    dp_source = source + "dp_"

    target = "http://www.semanticweb.org/SkillOA/2025/3/KBII/"
    class_target="class_"
    op_target = target + "op_"
    dp_target = target + "dp_"

    # Lade die Quellontologie
    source_graph = Graph()
    source_graph.parse(source_ontology_path, format="turtle")

    # Lade die Zielontologie
    target_graph = Graph()
    target_graph.parse(target_ontology_path, format="turtle")
    return OntologyData(
        source_ontology_path,target_ontology_path,source_graph, target_graph,
        source, class_source, op_source, dp_source,
        target, class_target, op_target, dp_target
    )
data = reload_source_target()
print(data.class_source)


http://www.semanticweb.org/SkillOA/2025/3/KBI/class_


In [19]:
from rdflib import Graph, RDF, RDFS, URIRef
import os

def apply_subclass_inference(graph):
    """
    Fügt rdf:type-Oberklassen hinzu, wenn eine Instanz bereits eine Unterklasse verwendet.
    """
    new_triples = []

    # Alle Instanzen mit einem Typ
    for instance, _, subclass in graph.triples((None, RDF.type, None)):
        # Alle Oberklassen dieser Klasse holen
        for _, _, superclass in graph.triples((subclass, RDFS.subClassOf, None)):
            # Prüfen, ob die Instanz bereits als Instanz der Oberklasse deklariert ist
            if (instance, RDF.type, superclass) not in graph:
                new_triples.append((instance, RDF.type, superclass))

    # Neue Tripel hinzufügen
    for triple in new_triples:
        graph.add(triple)
        print(f"➕ Hinzugefügt: {triple[0]} rdf:type {triple[2]}")

    print(f"✅ {len(new_triples)} neue Typ-Zuordnungen durch SubClass-Inferenz erzeugt.")

data = reload_source_target()
apply_subclass_inference(
    graph=data.source_graph,
)
print("source_path =", data.source_path)
print("source_graph =", type(data.source_graph))
data.source_graph.serialize(destination=data.source_path, format="turtle")

➕ Hinzugefügt: http://www.semanticweb.org/SkillOA/2025/3/KBI/ProposedCapability1ToFulfillTestServiceRequest1 rdf:type http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Capability
➕ Hinzugefügt: http://www.semanticweb.org/SkillOA/2025/3/KBI/ProposedCapability1ToFulfillTestServiceRequest2 rdf:type http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Capability
➕ Hinzugefügt: http://www.semanticweb.org/SkillOA/2025/3/KBI/ProposedCapability2ToFulfillTestServiceRequest2 rdf:type http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Capability
➕ Hinzugefügt: http://www.semanticweb.org/SkillOA/2025/3/KBI/TestParameter1 rdf:type http://www.semanticweb.org/SkillOA/2025/3/KBI/class_SkillParameter
✅ 4 neue Typ-Zuordnungen durch SubClass-Inferenz erzeugt.
source_path = C:\Users\Alexander Verkhov\Downloads\KB1_v0.5.ttl
source_graph = <class 'rdflib.graph.Graph'>


<Graph identifier=N79dd1408df004706b3e7b7ade97dbd2d (<class 'rdflib.graph.Graph'>)>

# Technical Process in source Graph erstellen

In [ ]:
from rdflib import RDF, URIRef

def migrate_capability_to_technical_process(graph, old_uri, new_uri, class_ns, op_ns, dp_ns):
    """
    Wandelt eine ProposedCapability-Instanz in eine neue TechnicalProcess-Instanz um.
    Nur ausgewählte Tripel werden übernommen und umbenannt.
    """
    triples = list(graph.triples((old_uri, None, None)))
    
    for s, p, o in triples:
        print(s, p, o)
        print("Vergleich:", str(p), "==", str(op_ns.hasSkill), "→", str(p) == str(op_ns.hasSkill))
        # Vergleiche URIs als Strings, um Probleme mit Namespace-Objekten zu umgehen
        if str(p) == str(RDF.type) and str(o) == str(class_ns.ProposedCapability):
            graph.add((new_uri, RDF.type, class_ns.TechnicalProcess))

        elif str(p) == str(op_ns.hasSkill):
            graph.add((new_uri, op_ns.containsSkill, o))

        elif str(p) == str(op_ns.canFulfill):
            graph.add((new_uri, op_ns.canBeRealizedBy, o))

        elif str(p) == str(dp_ns.hasSkillModel):
            graph.add((new_uri, dp_ns.hasSkillCombination, o))

    graph.serialize(destination=r"C:\Users\Alexander Verkhov\Downloads\KB1_v0.5.ttl", format="turtle")

    print(f"✅ {old_uri} wurde als {new_uri} → class:TechnicalProcess übernommen.")

data = reload_source_target()

old_uri = URIRef("http://www.semanticweb.org/SkillOA/2025/3/KBI/ProposedCapability1ToFulfillTestServiceRequest1")
new_uri = URIRef("http://www.semanticweb.org/SkillOA/2025/3/KBI/TechnicalProcess1ToFulfillTestServiceRequest1")

migrate_capability_to_technical_process(
    graph=data.source_graph,
    old_uri=old_uri,
    new_uri=new_uri,
    class_ns=data.class_source_ns,
    op_ns=data.op_source_ns,
    dp_ns=data.dp_source_ns
)


http://www.semanticweb.org/SkillOA/2025/3/KBI/ProposedCapability1ToFulfillTestServiceRequest1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/SkillOA/2025/3/KBI/class_Capability
Vergleich: http://www.w3.org/1999/02/22-rdf-syntax-ns#type == http://www.semanticweb.org/SkillOA/2025/3/KBI/op_hasSkill → False
http://www.semanticweb.org/SkillOA/2025/3/KBI/ProposedCapability1ToFulfillTestServiceRequest1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/SkillOA/2025/3/KBI/class_ProposedCapability
Vergleich: http://www.w3.org/1999/02/22-rdf-syntax-ns#type == http://www.semanticweb.org/SkillOA/2025/3/KBI/op_hasSkill → False
http://www.semanticweb.org/SkillOA/2025/3/KBI/ProposedCapability1ToFulfillTestServiceRequest1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#NamedIndividual
Vergleich: http://www.w3.org/1999/02/22-rdf-syntax-ns#type == http://www.semanticweb.org/SkillOA/2025/3/KBI/op_hasSkill → False
http://www

# Skills, TechnicalProcesses und SkillParameters und FieldDevices in KB2

In [ ]:
#Für Appr1
def copy_all_instance_triples(data,class_uri):
    #data = reload_source_target()
    query = f"""
        SELECT DISTINCT ?instance ?p ?o
        WHERE {{
            ?instance a <{class_uri}> .
            ?instance ?p ?o.
        }}
    """
    results = data.source_graph.query(query)
    df = pd.DataFrame(results, columns=[str(var) for var in results.vars])
    for index, row in df.iterrows():
        indiv_uri = str(row['instance'])
        pred_uri = str(row['p'])
        obj_uri = str(row['o'])
        if indiv_uri.startswith(data.source):
            new_indiv_uri = indiv_uri.replace(data.source,data.target)
        else:
            new_indiv_uri = indiv_uri
        new_class_uri = class_uri.replace(data.source,data.target)
        data.target_graph.add((URIRef(new_indiv_uri), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef(new_class_uri)))
        if pred_uri.startswith(data.source):
            new_pred_uri = pred_uri.replace(data.source,data.target)
        else:
            new_pred_uri = pred_uri

        if obj_uri.startswith("http://"):
            if obj_uri.startswith(data.source):
                new_obj_uri = obj_uri.replace(data.source,data.target)
            else:
               new_obj_uri = obj_uri 
               
            data.target_graph.add((URIRef(new_indiv_uri), URIRef(new_pred_uri), URIRef(new_obj_uri)))
            print(new_indiv_uri," ",new_pred_uri," ",new_obj_uri)
        else:
            new_value = obj_uri
            data.target_graph.add((URIRef(new_indiv_uri), URIRef(new_pred_uri), Literal(new_value)))
            print("Mit Wert")
            print(new_indiv_uri," ",new_pred_uri," ",new_value)

    data.target_graph.serialize(destination=data.target_path, format="turtle")


        #print(new_indiv_uri," ",new_pred_uri," ",new_obj_uri)


        #print((new_capability_uri, URIRef(op_target_ns + "is_SubFunction_of"), new_service_uri))
       # target_graph.add((new_capability_uri, URIRef(op_target_ns + "is_SubFunction_of"), new_service_uri))
data = reload_source_target()

for cls in ["Skill","SkillParameter","TechnicalProcess","FieldDevice"]:
    copy_all_instance_triples(data.class_source+cls)
#print(df2)

http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.w3.org/1999/02/22-rdf-syntax-ns#type   http://www.w3.org/2002/07/owl#NamedIndividual
http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.w3.org/1999/02/22-rdf-syntax-ns#type   http://www.semanticweb.org/SkillOA/2025/3/KBII/class_Skill
http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.semanticweb.org/SkillOA/2025/3/KBII/op_hasParallelSkill   http://www.semanticweb.org/SkillOA/2025/3/KBII/Skill_SenseWithLBS02
http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.semanticweb.org/SkillOA/2025/3/KBII/op_hasSubSkill   http://www.semanticweb.org/SkillOA/2025/3/KBII/Skill_MovePunchHeadUpWithA02
http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.semanticweb.org/SkillOA/2025/3/KBII/op_hasSubSkill   http://www.semanticweb.org/SkillOA/2025/3/KBII/Skill_SenseWithCS01
http://www.semanticweb.org/

In [112]:
#Für Appr2 mit Allem als DP
def add_skill_parameters(data,skill_uri):
    query_params = f""" 
        PREFIX cl: <{data.class_source}>
        PREFIX op: <{data.op_source}>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        SELECT DISTINCT ?property ?ParamIndiv ?dataProp ?datavalue WHERE {{
            <{skill_uri}> a cl:Skill;
                ?property ?ParamIndiv.
            ?property rdfs:subPropertyOf op:offersSkillParameter.
            FILTER regex(str(?property), "Parameter")
            ?ParamIndiv ?dataProp ?datavalue.
            ?dataProp a owl:DatatypeProperty.
        }}
    """
    results = data.source_graph.query(query_params)
    df = pd.DataFrame(results, columns=[str(var) for var in results.vars])
    new_skill_uri = skill_uri.replace(data.source, data.target)
    for index, row in df.iterrows():

        dp_uri = str(row['dataProp'])
        op_uri = str(row['property'])
        if  "ParameterIn" in op_uri:
            new_dp_uri = data.target+"dp_hasPositionRangeIn"
        elif  "ParameterOut" in op_uri:
            new_dp_uri = data.target+"dp_hasPositionRangeOut"
        else:
            new_dp_uri = dp_uri.replace(data.source,data.target)
        datV = str(row['datavalue'])

        data.target_graph.add((URIRef(new_skill_uri), URIRef(new_dp_uri), Literal(datV)))
        print(new_skill_uri," ",new_dp_uri," ",datV)

    data.target_graph.serialize(destination=data.target_path, format="turtle")
    

def add_field_device(data,skill_uri):
    query_fd = f""" 
        PREFIX cl: <{data.class_source}>
        PREFIX op: <{data.op_source}>
        PREFIX dp: <{data.dp_source}>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        SELECT DISTINCT ?fielddevice ?portaddr WHERE {{
            <{skill_uri}> a cl:Skill.
            ?fielddevice a cl:FieldDevice;
                op:providesAtomicSkill <{skill_uri}>;
                dp:hasPhysicalPortAddress ?portaddr
        }}
    """
    results = data.source_graph.query(query_fd)
    df = pd.DataFrame(results, columns=[str(var) for var in results.vars])
    new_skill_uri = skill_uri.replace(data.source, data.target)
    for index, row in df.iterrows():

        portaddr = str(row['portaddr'])
        dp_uri = data.dp_target+"hasFieldDevicePortAddress"

        data.target_graph.add((URIRef(new_skill_uri), URIRef(dp_uri), Literal(portaddr)))
        print(new_skill_uri," ",dp_uri," ",portaddr)

    data.target_graph.serialize(destination=data.target_path, format="turtle")

#data = reload_source_target()
#add_skill_parameters(data,"http://www.semanticweb.org/SkillOA/2025/3/KBI/TestSkill2")
#add_field_device(data,"http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_SenseWithLBS01")


def create_dp_from_indiv(data,class_uri):
    #data = reload_source_target()
    query = f"""
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        SELECT DISTINCT ?instance
        WHERE {{
            ?instance a <{class_uri}> .
            ?instance a owl:NamedIndividual.
        }}
    """
    results = data.source_graph.query(query)
    df = pd.DataFrame(results, columns=[str(var) for var in results.vars])
    for index, row in df.iterrows():
        indiv_uri = str(row['instance'])

        new_indiv_uri = indiv_uri.replace(data.source,data.target)
        new_class_uri = class_uri.replace(data.source,data.target)
        data.target_graph.add((URIRef(new_indiv_uri), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef(new_class_uri)))
        print(new_indiv_uri," ","a"," ",new_class_uri)

        add_skill_parameters(data, indiv_uri)
        add_field_device(data,indiv_uri)

    data.target_graph.serialize(destination=data.target_path, format="turtle")


        #print(new_indiv_uri," ",new_pred_uri," ",new_obj_uri)


        #print((new_capability_uri, URIRef(op_target_ns + "is_SubFunction_of"), new_service_uri))
       # target_graph.add((new_capability_uri, URIRef(op_target_ns + "is_SubFunction_of"), new_service_uri))
data = reload_source_target()
copy_all_instance_triples(data.class_source+"Skill")
create_dp_from_indiv(data, data.class_source+"Skill")
#print(df2)

http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.w3.org/1999/02/22-rdf-syntax-ns#type   http://www.w3.org/2002/07/owl#NamedIndividual
http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.w3.org/1999/02/22-rdf-syntax-ns#type   http://www.semanticweb.org/SkillOA/2025/3/KBII/class_Skill
http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.semanticweb.org/SkillOA/2025/3/KBII/op_hasParallelSkill   http://www.semanticweb.org/SkillOA/2025/3/KBII/Skill_SenseWithLBS02
http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.semanticweb.org/SkillOA/2025/3/KBII/op_hasSubSkill   http://www.semanticweb.org/SkillOA/2025/3/KBII/Skill_MovePunchHeadUpWithA02
http://www.semanticweb.org/SkillOA/2025/3/KBII/CSkill_PunchingProcess   http://www.semanticweb.org/SkillOA/2025/3/KBII/op_hasSubSkill   http://www.semanticweb.org/SkillOA/2025/3/KBII/Skill_SenseWithCS01
http://www.semanticweb.org/

In [108]:
query = f"""
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        SELECT DISTINCT ?instance
        WHERE {{
            ?instance a <{data.class_source+"Skill"}> .
            #?instance a owl:NamedIndividual.
        }}
    """
results = data.source_graph.query(query)
df = pd.DataFrame(results, columns=[str(var) for var in results.vars])

print(df)

                                                                               instance
0                  http://www.semanticweb.org/SkillOA/2025/3/KBI/CSkill_PunchingProcess
1   http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_MoveConveyorBeltBackwardWithA01
2    http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_MoveConveyorBeltForwardWithA01
3          http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_MovePunchHeadDownWithA02
4            http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_MovePunchHeadUpWithA02
5                     http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_SenseWithCS01
6                     http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_SenseWithCS02
7                    http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_SenseWithLBS01
8                    http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_SenseWithLBS02
9                              http://www.semanticweb.org/SkillOA/2025/3/KBI/TestSkill1
10                             h

In [100]:
def add_skill_parameters(data,skill_uri):
    query_params = f""" 
        PREFIX cl: <{data.class_source}>
        PREFIX op: <{data.op_source}>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        SELECT DISTINCT ?property ?ParamIndiv ?dataProp ?datavalue WHERE {{
            <{skill_uri}> a cl:Skill;
                ?property ?ParamIndiv.
            ?property rdfs:subPropertyOf op:offersSkillParameter.
            FILTER regex(str(?property), "Parameter")
            ?ParamIndiv ?dataProp ?datavalue.
            ?dataProp a owl:DatatypeProperty.
        }}
    """
    results = data.source_graph.query(query_params)
    df = pd.DataFrame(results, columns=[str(var) for var in results.vars])
    new_skill_uri = skill_uri.replace(data.source, data.target)
    for index, row in df.iterrows():

        dp_uri = str(row['dataProp'])
        op_uri = str(row['property'])
        if  "ParameterIn" in op_uri:
            new_dp_uri = data.target+"dp_hasPositionRangeIn"
        elif  "ParameterOut" in op_uri:
            new_dp_uri = data.target+"dp_hasPositionRangeOut"
        else:
            new_dp_uri = dp_uri.replace(data.source,data.target)
        datV = str(row['datavalue'])

        data.target_graph.add((URIRef(new_skill_uri), URIRef(new_dp_uri), Literal(datV)))
        print(new_skill_uri," ",new_dp_uri," ",datV)

    data.target_graph.serialize(destination=data.target_path, format="turtle")
    

def add_field_device(data,skill_uri):
    query_fd = f""" 
        PREFIX cl: <{data.class_source}>
        PREFIX op: <{data.op_source}>
        PREFIX dp: <{data.dp_source}>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        SELECT DISTINCT ?fielddevice ?portaddr WHERE {{
            <{skill_uri}> a cl:Skill.
            ?fielddevice a cl:FieldDevice;
                op:providesAtomicSkill <{skill_uri}>;
                dp:hasPhysicalPortAddress ?portaddr
        }}
    """
    results = data.source_graph.query(query_fd)
    df = pd.DataFrame(results, columns=[str(var) for var in results.vars])
    new_skill_uri = skill_uri.replace(data.source, data.target)
    for index, row in df.iterrows():

        portaddr = str(row['portaddr'])
        dp_uri = data.dp_target+"hasFieldDevicePortAddress"

        data.target_graph.add((URIRef(new_skill_uri), URIRef(dp_uri), Literal(portaddr)))
        print(new_skill_uri," ",dp_uri," ",portaddr)

    data.target_graph.serialize(destination=data.target_path, format="turtle")

data = reload_source_target()
add_skill_parameters(data,"http://www.semanticweb.org/SkillOA/2025/3/KBI/TestSkill2")
add_field_device(data,"http://www.semanticweb.org/SkillOA/2025/3/KBI/Skill_SenseWithLBS01")


http://www.semanticweb.org/SkillOA/2025/3/KBII/TestSkill2   http://www.semanticweb.org/SkillOA/2025/3/KBII/dp_hasFeatureType   TestType1
http://www.semanticweb.org/SkillOA/2025/3/KBII/TestSkill2   http://www.semanticweb.org/SkillOA/2025/3/KBII/dp_hasPositionRangeIn   [3;0;0]
http://www.semanticweb.org/SkillOA/2025/3/KBII/TestSkill2   http://www.semanticweb.org/SkillOA/2025/3/KBII/dp_hasPositionRangeOut   [3;0;0]
http://www.semanticweb.org/SkillOA/2025/3/KBII/Skill_SenseWithLBS01   http://www.semanticweb.org/SkillOA/2025/3/KBII/dp_hasFieldDevicePortAddress   xDI%1.2_A1_1
